In [ ]:
import React, { useState, useEffect } from 'react';
import { base44 } from '@/api/base44Client';
import { useQuery } from '@tanstack/react-query';
import { Card, CardContent, CardHeader, CardTitle } from "@/components/ui/card";
import { Button } from "@/components/ui/button";
import { Badge } from "@/components/ui/badge";
import { ArrowLeft, TrendingUp, Target, Award, Calendar } from 'lucide-react';
import { Link } from 'react-router-dom';
import { createPageUrl } from '../utils';
import { BarChart, Bar, LineChart, Line, PieChart, Pie, Cell, XAxis, YAxis, CartesianGrid, Tooltip, Legend, ResponsiveContainer } from 'recharts';
import { format, subDays, startOfWeek, endOfWeek } from 'date-fns';

export default function Analytics() {
  const [user, setUser] = useState(null);
  const [userProfile, setUserProfile] = useState(null);
  const [timeRange, setTimeRange] = useState('week'); // week, month

  useEffect(() => {
    const fetchUser = async () => {
      try {
        const currentUser = await base44.auth.me();
        setUser(currentUser);
        const profiles = await base44.entities.UserProfile.filter({ created_by: currentUser.email });
        if (profiles.length > 0) setUserProfile(profiles[0]);
      } catch (err) {
        console.error(err);
      }
    };
    fetchUser();
  }, []);

  const { data: scans = [] } = useQuery({
    queryKey: ['scans', user?.email],
    queryFn: () => base44.entities.FoodScan.filter({ created_by: user.email }, '-scan_date', 200),
    enabled: !!user,
    initialData: []
  });

  // Calculate weekly trends
  const getWeeklyData = () => {
    const daysOfWeek = [];
    for (let i = 6; i >= 0; i--) {
      const date = format(subDays(new Date(), i), 'yyyy-MM-dd');
      const dayScans = scans.filter(s => s.scan_date === date);
      const totalCalories = dayScans.reduce((sum, s) => sum + (s.calories || 0), 0);
      
      daysOfWeek.push({
        date: format(subDays(new Date(), i), 'EEE'),
        calories: Math.round(totalCalories),
        target: userProfile?.daily_calorie_target || 2000
      });
    }
    return daysOfWeek;
  };

  // Macro distribution
  const getMacroDistribution = () => {
    const totals = scans.reduce((acc, scan) => ({
      protein: acc.protein + (scan.protein || 0),
      carbs: acc.carbs + (scan.carbs || 0),
      fats: acc.fats + (scan.fats || 0)
    }), { protein: 0, carbs: 0, fats: 0 });

    return [
      { name: 'Protein', value: Math.round(totals.protein), color: '#3b82f6' },
      { name: 'Carbs', value: Math.round(totals.carbs), color: '#f59e0b' },
      { name: 'Fats', value: Math.round(totals.fats), color: '#a855f7' }
    ];
  };

  // Health score trend
  const getHealthScoreTrend = () => {
    return getWeeklyData().map((day, i) => {
      const date = format(subDays(new Date(), 6 - i), 'yyyy-MM-dd');
      const dayScans = scans.filter(s => s.scan_date === date);
      const avgScore = dayScans.length > 0 
        ? dayScans.reduce((sum, s) => sum + (s.health_score || 0), 0) / dayScans.length
        : 0;
      
      return {
        date: day.date,
        score: Math.round(avgScore * 10) / 10
      };
    });
  };

  // Stats
  const getTotalScans = () => scans.length;
  const getAvgHealthScore = () => {
    if (scans.length === 0) return 0;
    const total = scans.reduce((sum, s) => sum + (s.health_score || 0), 0);
    return (total / scans.length).toFixed(1);
  };
  const getWeeklyAvgCalories = () => {
    const weekData = getWeeklyData();
    const total = weekData.reduce((sum, d) => sum + d.calories, 0);
    return Math.round(total / 7);
  };

  const weeklyData = getWeeklyData();
  const macroData = getMacroDistribution();
  const healthTrend = getHealthScoreTrend();

  if (!user || !userProfile) {
    return <div className="min-h-screen flex items-center justify-center">Loading...</div>;
  }

  return (
    <div className="min-h-screen bg-gradient-to-br from-emerald-50 via-teal-50 to-cyan-50">
      {/* Header */}
      <div className="bg-white/80 backdrop-blur-sm border-b border-slate-200 sticky top-0 z-10">
        <div className="max-w-4xl mx-auto px-6 py-4 flex items-center gap-4">
          <Link to={createPageUrl('Home')}>
            <Button variant="ghost" size="icon">
              <ArrowLeft className="w-5 h-5" />
            </Button>
          </Link>
          <div>
            <h1 className="text-xl font-bold text-slate-900">Analytics</h1>
            <p className="text-sm text-slate-600">Track your nutrition journey</p>
          </div>
        </div>
      </div>

      <div className="max-w-4xl mx-auto px-6 py-8 space-y-6">
        {/* Quick Stats */}
        <div className="grid grid-cols-3 gap-4">
          <Card className="border-0 shadow-lg">
            <CardContent className="p-6 text-center">
              <div className="p-3 bg-blue-100 rounded-xl w-fit mx-auto mb-3">
                <Calendar className="w-6 h-6 text-blue-600" />
              </div>
              <p className="text-3xl font-bold text-slate-900">{getTotalScans()}</p>
              <p className="text-sm text-slate-600 mt-1">Total Scans</p>
            </CardContent>
          </Card>

          <Card className="border-0 shadow-lg">
            <CardContent className="p-6 text-center">
              <div className="p-3 bg-green-100 rounded-xl w-fit mx-auto mb-3">
                <Award className="w-6 h-6 text-green-600" />
              </div>
              <p className="text-3xl font-bold text-slate-900">{getAvgHealthScore()}</p>
              <p className="text-sm text-slate-600 mt-1">Avg Health Score</p>
            </CardContent>
          </Card>

          <Card className="border-0 shadow-lg">
            <CardContent className="p-6 text-center">
              <div className="p-3 bg-orange-100 rounded-xl w-fit mx-auto mb-3">
                <Target className="w-6 h-6 text-orange-600" />
              </div>
              <p className="text-3xl font-bold text-slate-900">{getWeeklyAvgCalories()}</p>
              <p className="text-sm text-slate-600 mt-1">Avg Daily Cal</p>
            </CardContent>
          </Card>
        </div>

        {/* Calorie Trend */}
        <Card className="border-0 shadow-lg">
          <CardHeader>
            <CardTitle className="flex items-center gap-2">
              <TrendingUp className="w-5 h-5 text-emerald-600" />
              Weekly Calorie Intake
            </CardTitle>
          </CardHeader>
          <CardContent>
            <ResponsiveContainer width="100%" height={250}>
              <BarChart data={weeklyData}>
                <CartesianGrid strokeDasharray="3 3" stroke="#e2e8f0" />
                <XAxis dataKey="date" stroke="#64748b" />
                <YAxis stroke="#64748b" />
                <Tooltip 
                  contentStyle={{ backgroundColor: '#fff', border: '1px solid #e2e8f0', borderRadius: '8px' }}
                />
                <Bar dataKey="calories" fill="#10b981" radius={[8, 8, 0, 0]} />
                <Bar dataKey="target" fill="#e2e8f0" radius={[8, 8, 0, 0]} />
              </BarChart>
            </ResponsiveContainer>
          </CardContent>
        </Card>

        {/* Macro Distribution */}
        <Card className="border-0 shadow-lg">
          <CardHeader>
            <CardTitle>Macronutrient Distribution</CardTitle>
          </CardHeader>
          <CardContent>
            <div className="flex items-center gap-8">
              <ResponsiveContainer width="50%" height={200}>
                <PieChart>
                  <Pie
                    data={macroData}
                    cx="50%"
                    cy="50%"
                    innerRadius={60}
                    outerRadius={80}
                    paddingAngle={5}
                    dataKey="value"
                  >
                    {macroData.map((entry, index) => (
                      <Cell key={`cell-${index}`} fill={entry.color} />
                    ))}
                  </Pie>
                  <Tooltip />
                </PieChart>
              </ResponsiveContainer>
              <div className="flex-1 space-y-3">
                {macroData.map((macro, idx) => (
                  <div key={idx} className="flex items-center justify-between">
                    <div className="flex items-center gap-3">
                      <div className="w-4 h-4 rounded" style={{ backgroundColor: macro.color }} />
                      <span className="text-sm font-medium text-slate-700">{macro.name}</span>
                    </div>
                    <span className="text-sm font-semibold text-slate-900">{macro.value}g</span>
                  </div>
                ))}
              </div>
            </div>
          </CardContent>
        </Card>

        {/* Health Score Trend */}
        <Card className="border-0 shadow-lg">
          <CardHeader>
            <CardTitle>Health Score Trend</CardTitle>
          </CardHeader>
          <CardContent>
            <ResponsiveContainer width="100%" height={200}>
              <LineChart data={healthTrend}>
                <CartesianGrid strokeDasharray="3 3" stroke="#e2e8f0" />
                <XAxis dataKey="date" stroke="#64748b" />
                <YAxis domain={[0, 10]} stroke="#64748b" />
                <Tooltip 
                  contentStyle={{ backgroundColor: '#fff', border: '1px solid #e2e8f0', borderRadius: '8px' }}
                />
                <Line 
                  type="monotone" 
                  dataKey="score" 
                  stroke="#10b981" 
                  strokeWidth={3}
                  dot={{ fill: '#10b981', r: 5 }}
                />
              </LineChart>
            </ResponsiveContainer>
          </CardContent>
        </Card>
      </div>
    </div>
  );
}

SyntaxError: invalid syntax (1337589283.py, line 1)